In [135]:
import pandas as pd
import pickle as pk
import numpy as np
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
from keras.applications.vgg16 import VGG16
import tensorflow as tf
from sklearn.metrics import accuracy_score

In [136]:
transfer=True

In [137]:
#(x_train, y_train), (x_test, y_test)= tf.keras.datasets.cifar100.load_data(label_mode="fine")
(ds_train,ds_val, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train[:80%]','train[80%:]','test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [138]:

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label
def normalize_transpose_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.transpose(tf.cast(image, tf.float32) / 255.), label
def pad_img(image, label):
    """turns 28 x 28 image into 32 x 32"""
    padded_im=tf.image.pad_to_bounding_box(image, 4, 4, 32, 32)
    tripled_im = tf.image.grayscale_to_rgb(padded_im)
    return tripled_im, label
def get_label(image,label):
    return label
def get_image(image,label):
    return image
# this just normalizes the image
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
if transfer:
    ds_train = ds_train.map(pad_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# cache and prefetch save memory 
ds_train = ds_train.cache()
# we shuffle data (reduces effects of order when training model)
ds_train = ds_train.shuffle(ds_info.splits['train[:80%]'].num_examples)
# batch size tells us how many samples are needed for stochastic gradient descent 
# (a small number like 32 is empirically found to be better)
ds_train = ds_train.batch(32)# batch of 32
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


ds_val = ds_val.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
if transfer:
    ds_val = ds_val.map(pad_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val = ds_val.cache()
ds_val = ds_val.shuffle(ds_info.splits['train[80%:]'].num_examples)
ds_val = ds_val.batch(32)# batch of 32
ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)


ds_test_label=ds_test.map(
    get_label, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test_images=ds_test.map(
    get_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

print(ds_test)
#image = ds_test.astype(np.float32)
#image = np.expand_dims(np.expand_dims(image, axis=2), axis=0)
#image = np.concatenate((image, 0.5*image), 0)
#image = np.concatenate((image, 0.5*image), 0)
#image3D = np.expand_dims(image, axis=0)
#image3D = np.concatenate((image3D, 0.5*image3D), 0)

ds_test_trans = ds_test.map(normalize_transpose_img)
ds_test_trans_images=ds_test_trans.map(get_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test_trans = ds_test_trans.batch(32)
ds_test_trans = ds_test_trans.cache()
ds_test_trans = ds_test_trans.prefetch(tf.data.experimental.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
if transfer:
    ds_test = ds_test.map(pad_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(32)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
print(ds_test)

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>
<PrefetchDataset shapes: ((None, 32, 32, 3), (None,)), types: (tf.float32, tf.int64)>


In [139]:
# this example model 
#    - first flattens data into a vector (image is 28 x 28)
#    - then creates a dense 128-node layer
#    - then creates a dropout "layer" (it says how many nodes are dropped out in previous layer)
#    - then another 128-node layer
#    - and finally 10-node layer as the head. 
# The max value of the head = the predicted image
#




if transfer:
    # now let's take advantage of a large model with a well-trained set of weights
    # load model
    #PTmodel = VGG16(include_top=False, weights='imagenet')
    # summarize the model
    #PTmodel.summary()
    #yhat = PTmodel.predict(image)
    inp = (32,32,3)
    VGG = VGG16(include_top=False,weights='imagenet',input_shape=inp)#,pooling='avg')
    # remove the output layer
    VGG = tf.keras.Model(inputs=VGG.inputs, outputs=VGG.layers[-16].output)
    #we choose not to train weights of VGG network, 
    #but if you play around with this model
    #you could if you wish
    #for layer in VGG.layers:
    #    layer.trainable = False 
       
    
    model = tf.keras.models.Sequential()
    model.add(VGG)
    #model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(500,activation='relu'))
    model.add(tf.keras.layers.Conv2D(40, kernel_size=3, activation='relu'))
    #model.add(tf.keras.layers.Dropout(.5, input_shape=(64,)))
    #model.add(tf.keras.layers.Dense(100,activation='relu'))
    #model.add(tf.keras.layers.Dropout(.5, input_shape=(32,)))
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(40,activation='relu'))
    #model.add(tf.keras.layers.Dropout(.5, input_shape=(32,)))
    model.add(tf.keras.layers.Dense(10))
    print(model.summary())
else:
    model = tf.keras.models.Sequential([])
    model.add(tf.keras.layers.Conv2D(100, kernel_size=3, activation='relu', input_shape=(28,28,1)))#64
    #model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(40, kernel_size=3, activation='relu'))
    #model.add(tf.keras.layers.BatchNormalization())
    #model.add(tf.keras.layers.Dropout(.5))
    #model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.BatchNormalization())
    #tf.keras.layers.Dense(24,activation='relu'),
    model.add(tf.keras.layers.Dense(10))
    print(model.summary())
# this specifies how we find the best NN
# - Optimizer like Adam is found to work well
# - Loss is "sparse categorical cross entropy" (you can choose whatever loss function on keras improves your model)
# - We also record accuracy ("metric"), this does not affect training
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
# this is stuff we record
# "early stopping" tells us when we found the optimum without training more epochs
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=1),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

# the training of the model
# we use ds_train data, and up to 30 epochs (less when Early Stopping is used)
# we also record callbacks, and we determine the optimal model by where validation is smallest
history=model.fit(
    ds_train,
    epochs=30,
    callbacks=my_callbacks,
    validation_data=ds_val
)


Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_26 (Functional)        (None, 16, 16, 64)        38720     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 40)        23080     
_________________________________________________________________
flatten_31 (Flatten)         (None, 7840)              0         
_________________________________________________________________
dense_86 (Dense)             (None, 10)                78410     
Total params: 140,210
Trainable params: 140,210
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
1500/1500 [==============================] - 92s 60ms/step - loss: 0.6641 - sparse_categorical_accuracy: 0.8147 - val_loss: 0.2976 - val_sparse_categorical_accuracy: 0.8938
Epoch 2/30
1500/1500 [==============================] - 

In [ ]:
# overall accuracy
# pre-trained model: best = 0.7399
# No-pre-training: best = 0.91

In [140]:
gt_labels=list(ds_test_label)
predictions=[np.argmax(im) for im in model.predict(ds_test)]
print(len(predictions))
print(accuracy_score(predictions,gt_labels))

10000
0.6634


In [141]:
from scipy.special import softmax
print(len(gt_labels))
print(len(ds_test))
label_names = ['t-shirt/top','trouser','pullover','dress','coat','sandals','shirt','sneaker','bag','ankle boots']
batch_size=32
for i in range(300):#len(ds_test)-1):
    rand_ind=np.random.randint(len(gt_labels))
    gt = gt_labels[rand_ind]
    pred = predictions[rand_ind]
    if pred != gt:
        batch_index = int(rand_ind/batch_size)
        within_batch_pos = rand_ind % batch_size
        dat=[im[:,:,0] for im in np.array(list(ds_test)[batch_index][0])]
    
        dat=np.array(dat).reshape(32,28,28)
        im = dat[within_batch_pos].reshape(28,28)
        tensored=tf.convert_to_tensor(dat.reshape(32,28,28,1))
        confidence = [np.max(softmax(v)) for v in model.predict([tensored])]
        plt.imshow(im,cmap='binary')
        plt.title('Predicted: '+label_names[pred]+' Actual: '+label_names[int(gt)])
        plt.axes().axis('off')
        plt.show()

10000
313


ValueError: cannot reshape array of size 32768 into shape (32,28,28)